In [ ]:
import subprocess
import zipfile, json, shutil, os, io
from google.colab import files, drive

# Mount Google Drive
drive.mount('/content/drive')

# Upload the broken model
print("Upload Liveness_Model_MobileNetV2_FFT.keras (ONE LAST TIME):")
uploaded = files.upload()
orig_path = list(uploaded.keys())[0]

FIXED_PATH = "/content/drive/MyDrive/Liveness_Model_Fixed.keras"

def patch_keras_model(src_path, dst_path):
    """Remove quantization_config from all layer configs inside the .keras zip."""
    import re

    with zipfile.ZipFile(src_path, 'r') as zin:
        names = zin.namelist()
        file_contents = {name: zin.read(name) for name in names}

    # The model config is stored in 'config.json'
    config_raw = file_contents.get("config.json")
    if config_raw is None:
        raise FileNotFoundError("config.json not found inside .keras file")

    config_str = config_raw.decode("utf-8")

    # Remove all occurrences of "quantization_config": null (and variants)
    config_str = re.sub(r',\s*"quantization_config"\s*:\s*null', '', config_str)
    config_str = re.sub(r'"quantization_config"\s*:\s*null,\s*', '', config_str)
    config_str = re.sub(r'"quantization_config"\s*:\s*null', '', config_str)

    file_contents["config.json"] = config_str.encode("utf-8")

    # Write patched zip
    with zipfile.ZipFile(dst_path, 'w', zipfile.ZIP_DEFLATED) as zout:
        for name, data in file_contents.items():
            zout.writestr(name, data)

    print(f"✅ Patched model saved to: {dst_path}")

patch_keras_model(orig_path, FIXED_PATH)

subprocess.run(["pip", "install", "-q", "keras>=3.4", "tensorflow>=2.17", "opencv-python-headless"], check=True)
print("✅ Done. Now go to Runtime > Restart session, then run from Cell 2.")

In [ ]:
# ─── CELL 2: Mount Drive & load model ──────────────────────
import numpy as np
import cv2
import keras
import io
from PIL import Image
from base64 import b64decode
from IPython.display import display, Javascript, HTML
from google.colab.output import eval_js
from google.colab import drive

drive.mount('/content/drive')

FIXED_PATH = "/content/drive/MyDrive/Liveness_Model_Fixed.keras"
model = keras.models.load_model(FIXED_PATH)
print("✅ Model loaded")
print(f"   Inputs : {[i.shape for i in model.inputs]}")
print(f"   Output : {model.output_shape}")


# ─── CELL 3: Preprocessing ──────────────────────────────────
IMG_H, IMG_W = 224, 224

def compute_fft_features(img_gray):
    f = np.fft.fft2(img_gray)
    fshift = np.fft.fftshift(f)
    magnitude = 20 * np.log(np.abs(fshift) + 1e-8)
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    return magnitude.astype(np.float32) / 255.0

def preprocess_frame(frame):
    img = cv2.resize(frame, (IMG_W, IMG_H))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fft_mag = compute_fft_features(img_gray)
    rgb_input = np.expand_dims(img_rgb, axis=0)
    fft_input = np.expand_dims(fft_mag, axis=(0, -1))
    return rgb_input, fft_input

def predict_liveness(frame):
    rgb_inp, fft_inp = preprocess_frame(frame)
    pred = model.predict([rgb_inp, fft_inp], verbose=0)
    score = float(pred[0][0])
    is_live = score > 0.5
    conf = score if is_live else 1 - score
    label = "LIVE ✅" if is_live else "SPOOF ❌"
    return label, conf, score


# ─── CELL 4: Webcam with BUTTON ─────────────────────────────
JS_WEBCAM_BUTTON = """
async function startWebcamWithButton() {
  // --- Layout ---
  const container = document.createElement('div');
  container.style.cssText = 'display:flex; flex-direction:column; align-items:center; gap:12px; font-family:sans-serif;';

  const video = document.createElement('video');
  video.style.cssText = 'width:400px; border-radius:10px; border:2px solid #ccc;';
  video.autoplay = true;

  const btn = document.createElement('button');
  btn.textContent = '📸 Capture Image';
  btn.style.cssText = `
    padding: 12px 28px;
    font-size: 16px;
    font-weight: bold;
    background: #4CAF50;
    color: white;
    border: none;
    border-radius: 8px;
    cursor: pointer;
    transition: background 0.2s;
  `;
  btn.onmouseover = () => btn.style.background = '#45a049';
  btn.onmouseout  = () => btn.style.background = '#4CAF50';

  const status = document.createElement('div');
  status.textContent = '📷 Camera ready — press the button when ready';
  status.style.cssText = 'color:#555; font-size:14px;';

  container.appendChild(video);
  container.appendChild(btn);
  container.appendChild(status);
  document.body.appendChild(container);

  // --- Start webcam ---
  const stream = await navigator.mediaDevices.getUserMedia({video: true});
  video.srcObject = stream;
  await video.play();

  // --- Wait for button click ---
  const dataUrl = await new Promise((resolve) => {
    btn.addEventListener('click', () => {
      btn.textContent = '⏳ Processing...';
      btn.disabled = true;
      btn.style.background = '#999';
      status.textContent = '✅ Image captured!';
      status.style.color = '#2e7d32';

      const canvas = document.createElement('canvas');
      canvas.width  = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      stream.getTracks().forEach(t => t.stop());
      video.style.opacity = '0.4';

      resolve(canvas.toDataURL('image/jpeg', 0.92));
    });
  });

  // Cleanup after short delay
  await new Promise(r => setTimeout(r, 1000));
  container.remove();

  return dataUrl;
}
"""

def capture_and_predict():
    display(Javascript(JS_WEBCAM_BUTTON))
    data_url = eval_js("startWebcamWithButton()")
    _, encoded = data_url.split(',', 1)
    pil_img = Image.open(io.BytesIO(b64decode(encoded))).convert('RGB')
    frame_bgr = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

    label, conf, raw_score = predict_liveness(frame_bgr)

    # Show result
    color = "#2e7d32" if "LIVE" in label else "#c62828"
    display(HTML(f"""
    <div style="font-family:sans-serif; padding:16px; border-radius:10px;
                border-left: 6px solid {color}; background:#f9f9f9; max-width:400px;">
      <h2 style="margin:0; color:{color};">{label}</h2>
      <p style="margin:6px 0; color:#333;">Confidence: <strong>{conf:.2%}</strong></p>
      <p style="margin:0; color:#888; font-size:13px;">Raw score: {raw_score:.4f} (threshold: 0.5)</p>
    </div>
    """))
    display(pil_img)


# ─── CELL 5: Run ────────────────────────────────────────────
capture_and_predict()


# ─── CELL 6 (OPTIONAL): Loop — keeps showing button each round ──
# import time
# for i in range(5):
#     print(f"\n--- Round {i+1} ---")
#     capture_and_predict()